In [16]:
import transformers
import datasets
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForMaskedLM

In [17]:
from torch.optim import AdamW
model: AutoModelForMaskedLM = AutoModelForMaskedLM.from_pretrained(
            'roberta-base')
optimizer: AdamW = AdamW(model.parameters(), lr=0.001)

In [31]:
model.config.to_dict()

{'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': None,
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': True,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'typical_p': 1.0,
 'repetition_penalty': 1.0,
 'length_penalty': 1.0,
 'no_repeat_ngram_size': 0,
 'encoder_no_repeat_ngram_size': 0,
 'bad_words_ids': None,
 'num_return_sequences': 1,
 'chunk_size_feed_forward': 0,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': None,
 'forced_eos_token_id': None,
 'remove_invalid_values': False,
 'exponential_decay_length_penalty': None,
 'su

In [22]:


path: str = "/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/data/processed/roberta_filtered/debug.txt"

dataset = load_dataset(
    "text", data_files={"train": path}, sample_by="line")

Found cached dataset text (/home/philko/.cache/huggingface/datasets/text/default-28d9029d0517ca66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6
    })
})

In [9]:
train_test = dataset['train'].train_test_split(test_size=0.01)
dataset = datasets.DatasetDict({
    'train': train_test['train'],
    'test': train_test['test']})
tokenize_function = lambda examples: tokenizer(examples["text"])
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [13]:
test = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [15]:
test['train'][0]

{'input_ids': [0,
  417,
  18045,
  56,
  450,
  475,
  12231,
  129,
  2330,
  11,
  39,
  7370,
  12,
  627,
  183,
  37,
  56,
  57,
  2421,
  8,
  5,
  183,
  37,
  376,
  184,
  31,
  5,
  1098,
  479,
  71,
  14,
  2156,
  37,
  56,
  295,
  75,
  57,
  2509,
  11,
  143,
  9,
  5,
  3493,
  8,
  5575,
  162,
  3494,
  1051,
  479,
  19,
  39,
  2038,
  1037,
  756,
  15,
  5,
  1430,
  2156,
  385,
  18045,
  56,
  295,
  75,
  770,
  7,
  28,
  26623,
  1329,
  19,
  5,
  9582,
  9,
  10,
  1928,
  479,
  1386,
  2156,
  37,
  770,
  7,
  1930,
  39,
  86,
  160,
  5,
  882,
  27865,
  454,
  70,
  722,
  9,
  5,
  363,
  479,
  37,
  129,
  1199,
  920,
  323,
  77,
  162,
  3494,
  3711,
  7,
  33,
  39,
  6729,
  28788,
  6555,
  479,
  2,
  0,
  119,
  12231,
  21,
  416,
  18446,
  160,
  5,
  12201,
  11,
  6958,
  8,
  2408,
  309,
  7,
  39,
  18559],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6
    })
})

In [13]:
dataset['train'][0]

{'text': "he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older . she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age . `` are n't you being a good boy ? '' she said . mason barely acknowledged her ."}

In [5]:
model_checkpoint = "roberta-base"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [70]:
tokenize_function = lambda examples: tokenizer(examples["text"])

In [72]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /home/philko/.cache/huggingface/datasets/text/default-28d9029d0517ca66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-8faa653c20f7c62b_*_of_00004.arrow


In [73]:
for sent in tokenized_datasets['train']['input_ids']:
    print(len(sent))

91
113
111
90
101
99


In [74]:
a = tokenized_datasets['train']['input_ids']

In [75]:
tokenizer.decode(tokenized_datasets["train"][0]["input_ids"])

"<s>he 'd seen the movie almost by mistake, considering he was a little young for the pg cartoon, but with older cousins, along with her brothers, mason was often exposed to things that were older. she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age. `` aren't you being a good boy? '' she said. mason barely acknowledged her.</s>"

In [12]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [77]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/6 [00:00<?, ? examples/s]

In [102]:
lm_datasets['train']['input_ids']

[[0,
  700,
  128,
  417,
  450,
  5,
  1569,
  818,
  30,
  5021,
  2156,
  2811,
  37,
  21,
  10,
  410,
  664,
  13,
  5,
  47194,
  15800,
  2156,
  53,
  19,
  2530,
  20005,
  2156,
  552,
  19,
  69,
  5396,
  2156,
  475,
  12231,
  21,
  747,
  4924,
  7,
  383,
  14,
  58,
  2530,
  479,
  79,
  6640,
  7,
  206,
  145,
  7501,
  30,
  3362,
  8,
  2530,
  1159,
  21,
  65,
  1219,
  596,
  37,
  21,
  10,
  215,
  10,
  205,
  1067,
  254,
  13,
  39,
  1046,
  479,
  45518,
  32,
  295,
  75,
  47,
  145,
  10,
  205,
  2143,
  17487,
  12801,
  79,
  26,
  479,
  475,
  12231,
  6254,
  5055,
  69,
  479,
  2,
  0,
  119,
  12231,
  21,
  416,
  18446,
  160,
  5,
  12201,
  11,
  6958,
  8,
  2408,
  309,
  7,
  39,
  18559,
  811,
  479,
  385,
  18045,
  56,
  450,
  475,
  12231,
  129,
  2330,
  11,
  39,
  7370,
  12,
  627,
  183,
  37,
  56,
  57,
  2421],
 [0,
  417,
  18045,
  56,
  450,
  475,
  12231,
  129,
  2330,
  11,
  39,
  7370,
  12,
  627,
  183,
  37

In [101]:
tokenizer.decode(lm_datasets["train"][0]["input_ids"]), tokenizer.decode(lm_datasets["train"][3]["input_ids"]),

IndexError: Invalid key: 3 is out of bounds for size 2

In [80]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [84]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [94]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=2,
    report_to="wandb"
)

PyTorch: setting up devices


In [95]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["train"],
    data_collator=data_collator,
)

In [96]:
trainer.train()

/home/philko/Documents/Uni/WiSe2223/UnsupervisedLearning/udl-negation/.venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2
  Number of trainable parameters = 124697433
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


  0%|          | 0/2 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 2
  Batch size = 8


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 8.568877220153809, 'eval_runtime': 1.273, 'eval_samples_per_second': 1.571, 'eval_steps_per_second': 0.786, 'epoch': 1.0}


***** Running Evaluation *****
  Num examples = 2
  Batch size = 8


  0%|          | 0/1 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 7.8370513916015625, 'eval_runtime': 1.3339, 'eval_samples_per_second': 1.499, 'eval_steps_per_second': 0.75, 'epoch': 2.0}
{'train_runtime': 12.9934, 'train_samples_per_second': 0.308, 'train_steps_per_second': 0.154, 'train_loss': 7.557348251342773, 'epoch': 2.0}


TrainOutput(global_step=2, training_loss=7.557348251342773, metrics={'train_runtime': 12.9934, 'train_samples_per_second': 0.308, 'train_steps_per_second': 0.154, 'train_loss': 7.557348251342773, 'epoch': 2.0})